In [45]:
import pandas as pd
import numpy as np


mc1-reports-data.csv fields:

    time: timestamp of incoming report/record, in the format YYYY-MM-DD hh:mm:ss

    location: id of neighborhood where person reporting is feeling the shaking and/or seeing the damage

    {shake_intensity, sewer_and_water, power, roads_and_bridges, medical, buildings}: reported categorical value of how violent the shaking was/how bad the damage was (0 - lowest, 10 - highest; missing data allowed)


In [46]:
df = pd.read_csv("mc1-reports-data.csv")
df.head()

,time,sewer_and_water,power,roads_and_bridges,medical,buildings,shake_intensity,location
0,2020-04-08 17:50:00,10.0,6.0,10.0,3.0,8.0,NaN,1
1,2020-04-09 13:50:00,2.0,10.0,0.0,8.0,4.0,0.0,1
2,2020-04-09 00:20:00,7.0,10.0,10.0,9.0,10.0,0.0,1
3,2020-04-08 17:25:00,1.0,1.0,2.0,10.0,7.0,NaN,1
4,2020-04-08 02:50:00,9.0,7.0,1.0,6.0,9.0,NaN,1


In [3]:
len(df)

83070

<h3>First I am trying to get an idea of data, min-max, count etc</h3>

In [4]:
df.describe()

,sewer_and_water,power,roads_and_bridges,medical,buildings,shake_intensity,location
count,82899.000000,83070.000000,83070.000000,35629.000000,82900.000000,70926.000000,83070.000000
mean,5.649139,6.045371,5.743289,5.322687,4.744005,2.682641,8.978488
std,2.787791,2.851951,2.506399,2.527679,2.256358,1.935366,5.123608
min,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,1.000000
25%,3.000000,4.000000,4.000000,4.000000,3.000000,1.000000,4.000000
50%,6.000000,7.000000,6.000000,6.000000,5.000000,2.000000,8.000000
75%,8.000000,8.000000,7.000000,7.000000,6.000000,4.000000,14.000000
max,10.000000,10.000000,10.000000,10.000000,10.000000,9.000000,19.000000


<h3> Now I noticed that medical, shake_intensity has unreported data</h3>
Now, the reason shake intensity is so low is probably because a lot of places were far away from the epicenter of the shake, so people didn't feel the shake as much and didn't report anything. But it is very easy to notice any changes in power supply and road_bridge damage thus they are always reported.

In [5]:
df.isna().sum()

time                     0
sewer_and_water        171
power                    0
roads_and_bridges        0
medical              47441
buildings              170
shake_intensity      12144
location                 0
dtype: int64

<h5>Now, I thought that we should check which location has more reports</h5>

In [6]:
df.groupby('location')['time'].count()

location
1      1662
2      5390
3     13535
4      2919
5      1710
6      3160
7       173
8     13889
9      8796
10     2213
11     2844
12     3463
13     1226
14     6429
15     4729
16     3145
17     1727
18     4545
19     1515
Name: time, dtype: int64

<h5>Location 8 and 3 have a very high amount of reports and they both are located near the border so that might be something we can look into. Location 7 is a forest so it makes sense that the number of reports from it is low.</h5>
To see the locations, check out the map given in file "About our city" in the neighborhood section.

<h5>We can seprate all the different columns and create graphs (bar graph or tree map etc) for each seprately. We'll also create a combined graph but this way we'll have more number of visualizations</h5>

In [7]:
df_sewer_and_water = df[['time','sewer_and_water','location']]
df_power= df[['time','power','location']]
df_roads_and_bridges = df[['time','roads_and_bridges','location']]
df_medical = df[['time','medical','location']]
df_buildings = df[['time','buildings','location']]
df_shake_intensity = df[['time','shake_intensity','location']]

In [8]:
df_power.head()

,time,power,location
0,2020-04-08 17:50:00,6.0,1
1,2020-04-09 13:50:00,10.0,1
2,2020-04-09 00:20:00,10.0,1
3,2020-04-08 17:25:00,1.0,1
4,2020-04-08 02:50:00,7.0,1


<h4>Before we start working on individual things, I think let's finalize aggregate stuff first</h4>
<h5>So I am thinking we can create a new aggregate dataframe with average report of a day for a location and use that to display the aggregate day's result</h5>

In [9]:
df_aggregate=df
df_aggregate.head()

,time,sewer_and_water,power,roads_and_bridges,medical,buildings,shake_intensity,location
0,2020-04-08 17:50:00,10.0,6.0,10.0,3.0,8.0,NaN,1
1,2020-04-09 13:50:00,2.0,10.0,0.0,8.0,4.0,0.0,1
2,2020-04-09 00:20:00,7.0,10.0,10.0,9.0,10.0,0.0,1
3,2020-04-08 17:25:00,1.0,1.0,2.0,10.0,7.0,NaN,1
4,2020-04-08 02:50:00,9.0,7.0,1.0,6.0,9.0,NaN,1


In [10]:
df_temp = df
df_temp = df_temp.fillna(0.0)
df_temp[['sewer_and_water','power','roads_and_bridges','medical','buildings','shake_intensity']].apply(abs)
df_temp[['date','time_of_day']] = df_temp.time.str.split(expand=True)
df_temp['hour'] = pd.to_datetime(df_temp['time_of_day'], format='%H:%M:%S').dt.hour

df_temp[['sewer_and_water_mean','power_mean','roads_and_bridges_mean','medical_mean','buildings_mean','shake_intensity_mean']] = df_temp.groupby(['location','hour','date'])[['sewer_and_water','power','roads_and_bridges','medical','buildings','shake_intensity']].transform('mean')
df_temp = df_temp.drop(['time','time_of_day','index','sewer_and_water','power','roads_and_bridges','medical','buildings','shake_intensity'],axis=1)
df_temp=df_temp.drop_duplicates(subset=['date','hour','location']).reset_index()
df_temp.sort_values(['hour', 'location'], ascending=[True, True],inplace=True)
df_temp = df_temp.drop('index',axis=1)
df_temp.head(20)

KeyError: "['index'] not found in axis"

In [ ]:
df_temp.to_csv('data_with_time.csv')

In [ ]:
df_shake_intensity[['date','time_of_day']] = df_shake_intensity.time.str.split(expand=True)
df_shake_intensity['hour'] = pd.to_datetime(df_shake_intensity['time_of_day'], format='%H:%M:%S').dt.hour
df_shake_intensity

/var/folders/_2/q6ygxk0j3rv7n_4_k8l_18gc0000gn/T/ipykernel_3334/1840806634.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_shake_intensity[['date','time_of_day']] = df_shake_intensity.time.str.split(expand=True)
/var/folders/_2/q6ygxk0j3rv7n_4_k8l_18gc0000gn/T/ipykernel_3334/1840806634.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_shake_intensity[['date','time_of_day']] = df_shake_intensity.time.str.split(expand=True)
/var/folders/_2/q6ygxk0j3rv7n_4_k8l_18gc0000gn/T/ipykernel_3334/1840806634

,time,shake_intensity,location,date,time_of_day,hour
0,2020-04-08 17:50:00,NaN,1,2020-04-08,17:50:00,17
1,2020-04-09 13:50:00,0.0,1,2020-04-09,13:50:00,13
2,2020-04-09 00:20:00,0.0,1,2020-04-09,00:20:00,0
3,2020-04-08 17:25:00,NaN,1,2020-04-08,17:25:00,17
4,2020-04-08 02:50:00,NaN,1,2020-04-08,02:50:00,2
...,...,...,...,...,...,...
83065,2020-04-10 02:30:00,2.0,8,2020-04-10,02:30:00,2
83066,2020-04-10 02:30:00,1.0,8,2020-04-10,02:30:00,2
83067,2020-04-09 16:45:00,1.0,8,2020-04-09,16:45:00,16
83068,2020-04-09 16:55:00,0.0,8,2020-04-09,16:55:00,16


In [ ]:
df_temp = df
df_temp = df_temp.fillna(0.0)
df_temp[['date','time_of_day']] = df_temp.time.str.split(expand=True)
df_temp = df_temp.drop(['time'],axis=1)
df_temp.head()

,sewer_and_water,power,roads_and_bridges,medical,buildings,shake_intensity,location,date,time_of_day
0,10.0,6.0,10.0,3.0,8.0,0.0,1,2020-04-08,17:50:00
1,2.0,10.0,0.0,8.0,4.0,0.0,1,2020-04-09,13:50:00
2,7.0,10.0,10.0,9.0,10.0,0.0,1,2020-04-09,00:20:00
3,1.0,1.0,2.0,10.0,7.0,0.0,1,2020-04-08,17:25:00
4,9.0,7.0,1.0,6.0,9.0,0.0,1,2020-04-08,02:50:00


In [ ]:
df_temp.to_csv("data_with_time.csv")

In [ ]:
df_aggregate[['date','time_of_day']] = df_aggregate.time.str.split(expand=True)
df_aggregate = df_aggregate.drop(['time','time_of_day'],axis=1)
df_aggregate.head()

,sewer_and_water,power,roads_and_bridges,medical,buildings,shake_intensity,location,date
0,10.0,6.0,10.0,3.0,8.0,NaN,1,2020-04-08
1,2.0,10.0,0.0,8.0,4.0,0.0,1,2020-04-09
2,7.0,10.0,10.0,9.0,10.0,0.0,1,2020-04-09
3,1.0,1.0,2.0,10.0,7.0,NaN,1,2020-04-08
4,9.0,7.0,1.0,6.0,9.0,NaN,1,2020-04-08


So, I am removing NAN for now, cause if they didn't report anything then they probably didn't feel anything

In [ ]:
df_aggregate = df_aggregate.fillna(0.0)
df_aggregate.head()

,sewer_and_water,power,roads_and_bridges,medical,buildings,shake_intensity,location,date
0,10.0,6.0,10.0,3.0,8.0,0.0,1,2020-04-08
1,2.0,10.0,0.0,8.0,4.0,0.0,1,2020-04-09
2,7.0,10.0,10.0,9.0,10.0,0.0,1,2020-04-09
3,1.0,1.0,2.0,10.0,7.0,0.0,1,2020-04-08
4,9.0,7.0,1.0,6.0,9.0,0.0,1,2020-04-08


In [ ]:
df_aggregate_mean = df_aggregate
df_aggregate_mean[['sewer_and_water_mean','power_mean','roads_and_bridges_mean','medical_mean','buildings_mean','shake_intensity_mean']] = df_aggregate.groupby(['location','date'])[['sewer_and_water','power','roads_and_bridges','medical','buildings','shake_intensity']].transform('mean')
df_aggregate_mean = df_aggregate_mean.drop(['sewer_and_water','power','roads_and_bridges','medical','buildings','shake_intensity'],axis=1)
df_aggregate_mean = df_aggregate_mean.drop_duplicates(subset=['date','location']).reset_index()
df_aggregate_mean = df_aggregate_mean.drop('index',axis=1)
df_aggregate_mean.head(20)

,location,date,sewer_and_water_mean,power_mean,roads_and_bridges_mean,medical_mean,buildings_mean,shake_intensity_mean
0,1,2020-04-08,5.015702,5.039669,5.019008,5.033884,5.113223,0.406612
1,1,2020-04-09,3.323383,3.273632,4.228856,4.099502,3.407960,0.129353
2,1,2020-04-10,4.608696,5.086957,4.869565,5.884058,5.753623,0.086957
3,1,2020-04-07,5.116883,5.506494,5.233766,6.480519,5.571429,0.129870
4,1,2020-04-06,4.085714,3.609524,4.076190,4.266667,4.323810,0.133333
5,2,2020-04-10,5.348018,5.162996,5.013216,0.026432,6.189427,0.074890
6,2,2020-04-09,3.474882,5.037915,4.318483,0.090995,1.898578,0.837915
7,2,2020-04-06,3.158974,3.010256,3.307692,0.092308,3.648718,0.148718
8,2,2020-04-08,3.153140,4.108689,5.058503,0.088902,1.285632,1.677086
9,2,2020-04-07,5.589520,5.497817,4.934498,0.139738,5.423581,0.087336


Removing the date 2020-04-11 as not all locations reported that day, the better thing to do would be to add 2020-04-11 with value of 0.0 to all the locations without that date, but this is easier for now.

In [ ]:
df_aggregate_mean = df_aggregate_mean[df_aggregate_mean.date != "2020-04-11"]
df_aggregate_mean.head(20)

,location,date,sewer_and_water_mean,power_mean,roads_and_bridges_mean,medical_mean,buildings_mean,shake_intensity_mean
0,1,2020-04-08,5.015702,5.039669,5.019008,5.033884,5.113223,0.406612
1,1,2020-04-09,3.323383,3.273632,4.228856,4.099502,3.407960,0.129353
2,1,2020-04-10,4.608696,5.086957,4.869565,5.884058,5.753623,0.086957
3,1,2020-04-07,5.116883,5.506494,5.233766,6.480519,5.571429,0.129870
4,1,2020-04-06,4.085714,3.609524,4.076190,4.266667,4.323810,0.133333
5,2,2020-04-10,5.348018,5.162996,5.013216,0.026432,6.189427,0.074890
6,2,2020-04-09,3.474882,5.037915,4.318483,0.090995,1.898578,0.837915
7,2,2020-04-06,3.158974,3.010256,3.307692,0.092308,3.648718,0.148718
8,2,2020-04-08,3.153140,4.108689,5.058503,0.088902,1.285632,1.677086
9,2,2020-04-07,5.589520,5.497817,4.934498,0.139738,5.423581,0.087336


In [ ]:
df_aggregate_mean.to_csv('df_aggregate_mean.csv')

We can create a visualization for just locations, like during the whole saga what was the average response for each location.

In [ ]:
df_aggregate_mean_without_date = df_aggregate
df_aggregate_mean_without_date[['sewer_and_water_mean','power_mean','roads_and_bridges_mean','medical_mean','buildings_mean','shake_intensity_mean']] = df_aggregate.groupby(['location'])[['sewer_and_water','power','roads_and_bridges','medical','buildings','shake_intensity']].transform('mean')
df_aggregate_mean_without_date = df_aggregate_mean_without_date.drop(['date','sewer_and_water','power','roads_and_bridges','medical','buildings','shake_intensity'],axis=1)
df_aggregate_mean_without_date = df_aggregate_mean_without_date.drop_duplicates(subset=['location']).reset_index()
df_aggregate_mean_without_date = df_aggregate_mean_without_date.drop('index',axis=1)
df_aggregate_mean_without_date

,location,sewer_and_water_mean,power_mean,roads_and_bridges_mean,medical_mean,buildings_mean,shake_intensity_mean
0,1,4.740072,4.759326,4.867629,4.974729,4.904934,0.329723
1,2,3.413544,4.313544,4.779777,0.089054,1.960111,1.266605
2,3,7.275434,8.461396,7.267085,7.055264,5.831991,4.590100
3,4,5.468654,4.224049,4.213772,0.096608,3.692018,4.263789
4,5,4.205263,5.065497,4.083626,2.110526,3.494737,0.519298
5,6,2.937975,3.062975,3.318987,3.306013,3.516139,0.766139
6,7,0.098266,7.838150,5.919075,0.034682,0.098266,4.421965
7,8,7.499748,7.457772,7.658291,0.087551,5.781986,1.527252
8,9,7.456003,5.726353,6.591291,5.184970,6.168486,0.973511
9,10,5.676909,7.618165,5.950294,0.075915,6.351559,1.372797


In [ ]:
def creating_json(df):
   entries = []
   values = set(df.iloc[:, 0])
   for v in range(len(df)):
      entries.append({
         'Name':df.iloc[v][0],
         'Children':[{'Name':'sewer_and_water_mean','value':df.iloc[v][1]},
                     {'Name':'power_mean','value':df.iloc[v][2]},
                     {'Name':'roads_and_bridges_mean','value':df.iloc[v][3]},
                     {'Name':'medical_mean','value':df.iloc[v][4]},
                     {'Name':'buildings_mean','value':df.iloc[v][5]},
                     {'Name':'shake_intensity_mean','value':df.iloc[v][6]}]
      })
      #print(df.iloc[18][1])
   return entries

In [ ]:
mydict = {"Name": "Himark",
          "Children": creating_json(df_aggregate_mean_without_date)}
mydict['Children']

[{'Name': 1.0,
  'Children': [{'Name': 'sewer_and_water_mean', 'value': 4.740072202166065},
   {'Name': 'power_mean', 'value': 4.759326113116727},
   {'Name': 'roads_and_bridges_mean', 'value': 4.8676293622142},
   {'Name': 'medical_mean', 'value': 4.974729241877256},
   {'Name': 'buildings_mean', 'value': 4.904933814681107},
   {'Name': 'shake_intensity_mean', 'value': 0.3297232250300842}]},
 {'Name': 2.0,
  'Children': [{'Name': 'sewer_and_water_mean', 'value': 3.413543599257885},
   {'Name': 'power_mean', 'value': 4.313543599257885},
   {'Name': 'roads_and_bridges_mean', 'value': 4.779777365491651},
   {'Name': 'medical_mean', 'value': 0.08905380333951762},
   {'Name': 'buildings_mean', 'value': 1.9601113172541744},
   {'Name': 'shake_intensity_mean', 'value': 1.2666048237476808}]},
 {'Name': 3.0,
  'Children': [{'Name': 'sewer_and_water_mean', 'value': 7.275434059844847},
   {'Name': 'power_mean', 'value': 8.461396379756188},
   {'Name': 'roads_and_bridges_mean', 'value': 7.2670853

In [ ]:
import json
with open("df_aggregate_mean_without_date.json", "w") as outfile:
    json.dump(mydict, outfile)

In [ ]:
df_power_plant = df[df['location']==4]
df_power_plant = df_power_plant.fillna(0.0)
df_power_plant[['date','time_of_day']] = df_power_plant.time.str.split(expand=True)
df_power_plant = df_power_plant.drop('time',axis=1)
df_power_plant


,sewer_and_water,power,roads_and_bridges,medical,buildings,shake_intensity,location,date,time_of_day
2347,4.0,0.0,8.0,0.0,7.0,0.0,4,2020-04-07,20:40:00
2348,5.0,8.0,3.0,0.0,5.0,0.0,4,2020-04-09,14:05:00
2349,9.0,10.0,0.0,0.0,2.0,0.0,4,2020-04-09,12:05:00
2350,5.0,10.0,0.0,0.0,9.0,1.0,4,2020-04-08,16:35:00
2351,3.0,2.0,6.0,0.0,5.0,0.0,4,2020-04-08,16:55:00
...,...,...,...,...,...,...,...,...,...
69621,7.0,6.0,5.0,0.0,7.0,5.0,4,2020-04-09,15:45:00
69622,8.0,5.0,6.0,0.0,4.0,5.0,4,2020-04-09,18:35:00
69623,7.0,5.0,5.0,0.0,5.0,6.0,4,2020-04-09,15:45:00
69624,6.0,6.0,6.0,0.0,4.0,6.0,4,2020-04-09,17:05:00


In [36]:
df_power_plant.to_csv("power_plant_data.csv")

NameError: name 'df_power_plant' is not defined

In [47]:
df_line = df
df_line = df_line.fillna(0.0)
df_line[['date','time_of_day']] = df_line.time.str.split(expand=True)
df[['sewer_and_water','power','roads_and_bridges','medical','buildings','shake_intensity']] = df_line[['sewer_and_water','power','roads_and_bridges','medical','buildings','shake_intensity']].abs

df_line

,time,sewer_and_water,power,roads_and_bridges,medical,buildings,shake_intensity,location,date,time_of_day
0,2020-04-08 17:50:00,10.0,6.0,10.0,3.0,8.0,0.0,1,2020-04-08,17:50:00
1,2020-04-09 13:50:00,2.0,10.0,0.0,8.0,4.0,0.0,1,2020-04-09,13:50:00
2,2020-04-09 00:20:00,7.0,10.0,10.0,9.0,10.0,0.0,1,2020-04-09,00:20:00
3,2020-04-08 17:25:00,1.0,1.0,2.0,10.0,7.0,0.0,1,2020-04-08,17:25:00
4,2020-04-08 02:50:00,9.0,7.0,1.0,6.0,9.0,0.0,1,2020-04-08,02:50:00
...,...,...,...,...,...,...,...,...,...,...
83065,2020-04-10 02:30:00,9.0,10.0,10.0,0.0,7.0,2.0,8,2020-04-10,02:30:00
83066,2020-04-10 02:30:00,8.0,10.0,10.0,0.0,7.0,1.0,8,2020-04-10,02:30:00
83067,2020-04-09 16:45:00,10.0,9.0,10.0,0.0,8.0,1.0,8,2020-04-09,16:45:00
83068,2020-04-09 16:55:00,8.0,8.0,9.0,0.0,7.0,0.0,8,2020-04-09,16:55:00


In [48]:
df_line.to_csv('df_line.csv')